# Bibliotecas

In [ ]:
!pip install TTS==0.15.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 762.2/762.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) .

In [ ]:
import os
import glob
import zipfile
from google.colab import drive
import pandas as pd
import warnings

from TTS.config.shared_configs import BaseAudioConfig, BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples

from trainer import Trainer, TrainerArgs
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

warnings.filterwarnings('ignore')

# Base de dados

In [ ]:
# Drive para chave API do Kaggle
drive.mount('/content/drive', force_remount=True)
drive_path = '/content/drive/MyDrive/'
kaggle_json_file = 'kaggle.json'

# Download do dataset diretamente do Kaggle
!mkdir -p ~/.kaggle
!cp "{drive_path}/{kaggle_json_file}" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mediatechlab/gneutralspeech

zip_file = 'gneutralspeech.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()
os.remove(zip_file)

Mounted at /content/drive
100% 4.75G/4.76G [00:47<00:00, 183MB/s]
100% 4.76G/4.76G [00:47<00:00, 108MB/s]


In [ ]:
# Carregar o arquivo CSV
df_voz = pd.read_csv("/content/voz_base_44kHz_16bit/metadata_voz_base_norm.csv", delimiter="|", header=None, engine='python', error_bad_lines=False)

# Criar um novo dataframe usando as colunas desejadas e separando-as com " | "
metadata_voz_df = df_voz[[0, 1]].apply(lambda x: ' | '.join(x.astype(str)), axis=1)

# Salvar o dataframe no formato de arquivo .txt
with open('/content/metadata_voz.txt', 'w') as f:
    for line in metadata_voz_df:
        f.write(f"{line}\n")

files_path = [filename.split('/')[-1].split('.')[0] for filename in glob.glob('/content/voz_base_44kHz_16bit/wavs/*.wav')]
lines_list = []

# Formatar os nomes dos arquivos na base de metadados
with open('/content/metadata_voz.txt', 'r') as metadata:
  for line in metadata.readlines():
    filename, txt = line.strip().split('|')
    filename = filename.strip()
    txt = txt.strip()
    if filename in files_path:
      lines_list.append('|'.join([filename, txt])+'\n')

# Escrever um novo arquivo de metadado corrigido
with open('/content/metadata.txt', 'w') as new_metadata:
  for line in lines_list[:800]:
    new_metadata.write(line)

In [ ]:
# Configurações de dataset
dataset_config = BaseDatasetConfig(formatter='thorsten', meta_file_train="/content/metadata.txt", path="/content/voz_base_44kHz_16bit")

# Diretório de saída
output_path = '/content/output'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Treinamento do GlowTTS (modelo acústico)

In [ ]:
# Configurações dos áudios
audio_config = BaseAudioConfig(
    sample_rate=44100,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

# Parâmetros para o treinamento do  modelo
config = GlowTTSConfig(
    audio=audio_config,
    batch_size=16,
    eval_batch_size=1,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="pt",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=50,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
)

# Pré-processamento dos áudios
ap = AudioProcessor.init_from_config(config)

# Tokenização
tokenizer, config = TTSTokenizer.init_from_config(config)

# Carregamento dos dados de treino e validação
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# Inicialização do modelo
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# Treinamento do modelo
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model,
    train_samples=train_samples, eval_samples=eval_samples
)
trainer.fit()

 > Setting up Audio Processor...
 | > sample_rate:44100
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 | > Found 800 files in /content/voz_base_44kHz_16bit


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/output/run-August-12-2023_11+43PM-0000000

 > Model has 28610257 parameters

 > EPOCH: 0/100
 --> /content/output/run-August-12-2023_11+43PM-0000000


[*] Pre-computing phonemes...


 40%|████      | 317/792 [00:04<00:07, 65.94it/s]

. . , , , … .
 [!] Character '…' not found in the vocabulary. Discarding it.


...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.003935609545026507 (+0.0)
     | > avg_loss: 19.032914297921316 (+0.0)
     | > avg_log_mle: 0.49319384353501455 (+0.0)
     | > avg_loss_dur: 18.539720262799943 (+0.0)

 > BEST MODEL : /content/output/run-August-12-2023_11+43PM-0000000/best_model_1950.pth

 > EPOCH: 39/100
 --> /content/output/run-August-12-2023_11+43PM-0000000

 > TRAINING (2023-08-13 00:58:59) 

   --> TIME: 2023-08-13 00:59:03 -- STEP: 0/50 -- GLOBAL_STEP: 1950
     | > loss: 16.12113380432129  (16.12113380432129)
     | > log_mle: 0.4928693175315857  (0.4928693175315857)
     | > loss_dur: 15.628265380859375  (15.628265380859375)
     | > amp_scaler: 4096.0  (4096.0)
     | > grad_norm: tensor(17.0339, device='cuda:0')  (tensor(17.0339, device='cuda:0'))
     | > current_lr: 9.75e-06 
     | > step_time: 1.6721  (1.6720740795135498)
     | > loader_time: 3.1406  (3.140556812286377)


 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.002169030053

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0021690300532749723 (+0.0)
     | > avg_loss: 18.8566677910941 (+0.0)
     | > avg_log_mle: 0.48236291323389324 (+0.0)
     | > avg_loss_dur: 18.374304907662527 (+0.0)

 > BEST MODEL : /content/output/run-August-12-2023_11+43PM-0000000/best_model_2000.pth

 > EPOCH: 40/100
 --> /content/output/run-August-12-2023_11+43PM-0000000

 > TRAINING (2023-08-13 01:00:57) 

   --> TIME: 2023-08-13 01:01:02 -- STEP: 0/50 -- GLOBAL_STEP: 2000
     | > loss: 15.78085994720459  (15.78085994720459)
     | > log_mle: 0.48144271969795227  (0.48144271969795227)
     | > loss_dur: 15.299417495727539  (15.299417495727539)
     | > amp_scaler: 4096.0  (4096.0)
     | > grad_norm: tensor(17.0037, device='cuda:0')  (tensor(17.0037, device='cuda:0'))
     | > current_lr: 9.999999999999999e-06 
     | > step_time: 1.4412  (1.4411780834197998)
     | > loader_time: 3.4323  (3.4323296546936035)


 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time:

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0029793807438441683 (+0.0)
     | > avg_loss: 18.617325919015066 (+0.0)
     | > avg_log_mle: 0.47254463178770884 (+0.0)
     | > avg_loss_dur: 18.144781521388463 (+0.0)

 > BEST MODEL : /content/output/run-August-12-2023_11+43PM-0000000/best_model_2050.pth

 > EPOCH: 41/100
 --> /content/output/run-August-12-2023_11+43PM-0000000

 > TRAINING (2023-08-13 01:02:55) 

   --> TIME: 2023-08-13 01:02:59 -- STEP: 0/50 -- GLOBAL_STEP: 2050
     | > loss: 15.199819564819336  (15.199819564819336)
     | > log_mle: 0.4708048105239868  (0.4708048105239868)
     | > loss_dur: 14.72901439666748  (14.72901439666748)
     | > amp_scaler: 8192.0  (8192.0)
     | > grad_norm: tensor(16.4805, device='cuda:0')  (tensor(16.4805, device='cuda:0'))
     | > current_lr: 1.025e-05 
     | > step_time: 1.5396  (1.5395569801330566)
     | > loader_time: 3.2339  (3.2339329719543457)

